In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import re
import lightgbm as lgb
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


        
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

df_train = pd.read_csv(r'C:\Users\higaj\Desktop\py\used_cars\train.csv')
df_test = pd.read_csv(r'C:\Users\higaj\Desktop\py\used_cars\test.csv')

df_train['fuel_type'].fillna('special', inplace=True)
# Transform model year to car_age
df_train['car_age'] = 2024 - df_train['model_year'] 
df_test['car_age'] = 2024 - df_test['model_year'] 

df_train.drop(columns='model_year', inplace=True)
df_test.drop(columns='model_year', inplace=True)


### Drop ID column
df_train.drop(columns='id', inplace=True)
df_test.drop(columns='id', inplace=True)


### Clean the 'Accident' column
df_train.loc[df_train['accident'] == 'None reported', 'accidentbool'] = False
df_train.loc[df_train['accident'] == 'At least 1 accident or damage reported', 'accidentbool'] = True
df_train.drop(columns='accident', inplace = True)

df_test.loc[df_test['accident'] == 'None reported', 'accidentbool'] = False
df_test.loc[df_test['accident'] == 'At least 1 accident or damage reported', 'accidentbool'] = True
df_test.drop(columns='accident', inplace = True)


### Clean the 'clean_title' column
df_train.loc[df_train['clean_title'] == 'Yes', 'cleanbool'] = True
df_train.drop(columns='clean_title', inplace = True)

df_test.loc[df_test['clean_title'] == 'Yes', 'cleanbool'] = True
df_test.drop(columns='clean_title', inplace = True)



############ Clean the 'Engine' column
def hp_extract(row):
    if 'HP' in row:
        hpower, rest = row.split('HP', 1)
        return pd.Series([hpower.strip(), rest.strip()])
    else:
        return pd.Series([np.nan, row])

def litters_extract(row):
    pattern = r'(\d+(?:\.\d+)?L)'  # Regex to capture only the liter value (integer or decimal)
    match = re.search(pattern, row)  # Search for the pattern in the row
    
    if match:  # If a match is found
        litrage = match.group(1).strip()  # Extract the matched liter value (e.g., '4.75L' or '2L')
        rest = row.replace(litrage, '').strip()  # Remove the matched liter value from the row
        return pd.Series([litrage, rest])  # Return the extracted liter and remaining text
    else:
        return pd.Series([np.nan, row])  # If no match, return NaN for litrage

def litters_extract2(row):
    if 'Liter' in row:
        hpower, rest = row.split('Liter', 1)

        return pd.Series([hpower.strip(), rest.strip()])
    else:
        return pd.Series([np.nan, row])
    
def get_fuel(fuel, row):
    if fuel in row:
        eng = row.replace(fuel, '').strip()
        return eng
    else:
        return row  
                      
    
# Keep this in order! Bc working in order stripping the texts
df_train[['horse_power', 'engine']] = df_train['engine'].apply(hp_extract)
df_train[['litrage', 'engine']] = df_train['engine'].apply(litters_extract)
df_train[['litrage2', 'engine']] = df_train['engine'].apply(litters_extract2)

df_train['litrage'] = df_train['litrage'].astype(str).str.rstrip('L')
df_train['litrage'] = df_train['litrage'].astype(float)
df_train['litrage2'] = df_train['litrage2'].astype(float)

df_train['litrage'] = df_train['litrage'].fillna(df_train['litrage2'])
df_train.drop(columns='litrage2', inplace = True)
#############################################################################

# Now, stripping the fuel type. There is already a 'fuel' column!

fueltp_list = ['Gasoline Fuel', 'Gasoline/Mild Electric Hybrid', 'Diesel Fuel', 'Gas/Electric Hybrid', 'Flex Fuel Capability',
              'Flexible Fuel', 'Diesel', 'Plug-In Electric/Gas', 'Hybrid']

for fuel in fueltp_list:
    df_train['engine'] = df_train['engine'].apply(lambda row: get_fuel(fuel, row))

## Change type of horse_power and litrage
df_train['horse_power'] = df_train['horse_power'].astype(float)
df_train['litrage'] = df_train['litrage'].astype(float)



# Keep this in order! Bc working in order stripping the texts
df_test[['horse_power', 'engine']] = df_test['engine'].apply(hp_extract)
df_test[['litrage', 'engine']] = df_test['engine'].apply(litters_extract)
df_test[['litrage2', 'engine']] = df_test['engine'].apply(litters_extract2)

df_test['litrage'] = df_test['litrage'].astype(str).str.rstrip('L')
df_test['litrage'] = df_test['litrage'].astype(float)
df_test['litrage2'] = df_test['litrage2'].astype(float)

df_test['litrage'] = df_test['litrage'].fillna(df_test['litrage2'])
df_test.drop(columns='litrage2', inplace = True)
#############################################################################

# Now, stripping the fuel type. There is already a 'fuel' column!

fueltp_list = ['Gasoline Fuel', 'Gasoline/Mild Electric Hybrid', 'Diesel Fuel', 'Gas/Electric Hybrid', 'Flex Fuel Capability',
              'Flexible Fuel', 'Diesel', 'Plug-In Electric/Gas', 'Hybrid']

for fuel in fueltp_list:
    df_test['engine'] = df_test['engine'].apply(lambda row: get_fuel(fuel, row))

## Change type of horse_power and litrage
df_test['horse_power'] = df_test['horse_power'].astype(float)
df_test['litrage'] = df_test['litrage'].astype(float)


brands_list = df_train['brand'].unique()
results = []
for brand in brands_list:

    bavgprice = df_train.loc[df_train['brand'] == brand, 'price'].mean()
    
    results.append({'brand': brand, 'avg_price': bavgprice})

avg_price_df = pd.DataFrame(results)

avg_price_df.loc[avg_price_df['avg_price'] > 0, 'brand_range'] = '1'
avg_price_df.loc[avg_price_df['avg_price'] > 26000, 'brand_range'] = '2'
avg_price_df.loc[avg_price_df['avg_price'] > 40000, 'brand_range'] = '3'
avg_price_df.loc[avg_price_df['avg_price'] > 50000, 'brand_range'] = '4'
avg_price_df.loc[avg_price_df['avg_price'] > 70000, 'brand_range'] = '5'



avg_price_df.sort_values(by='avg_price', ascending=False)

def funct(row):
    aux = avg_price_df.loc[avg_price_df['brand'] == row, 'brand_range'].values[0]
    return aux

df_train['brand_type'] = df_train['brand'].apply(funct)
df_test['brand_type'] = df_test['brand'].apply(funct)


############# NEW CODE##################################################################################################################################

def extract_transmission(row):
    split_patterns = ['-Speed', '-SPEED', ' Speed']
    
    for pattern in split_patterns:
        if pattern in row:
            outp = row.split(pattern)
            return outp
    return [np.nan, row]

df_test[['speed', 'transrest']] = df_test['transmission'].apply(extract_transmission).apply(pd.Series)
df_train[['speed', 'transrest']] = df_train['transmission'].apply(extract_transmission).apply(pd.Series)
df_train.loc[df_train['speed'] == 'Single', 'speed'] = 1 ## "Single Speed -> 1 "
df_test.loc[df_train['speed'] == 'Single', 'speed'] = 1 ## "Single Speed -> 1 "


automatic = ['A/T', ' A/T',' Automatic', 'Automatic', ' AT', ' Automatic with Overdrive', ' DCT Automatic',]
manual = [' Manual', ' M/T', 'M/T', ' Mt', 'Manual']
nicer = ['Transmission w/Dual Shift Mode', '',
       ' Automatic with Auto-Shift', 'CVT Transmission',
       'Automatic CVT',
       'Transmission Overdrive Switch',
       ' Electronically Controlled Automatic with O', 'F', 'CVT-F', 
         '–', '2', ' At/Mt', '', 'Variable', ' Fixed Gear', 'SCHEDULED FOR OR IN PRODUCTION']
def classify_transmission(row):
    if row in automatic:
        return 3
    elif row in manual:
        return 2
    elif row in nicer:
        return 1
df_train['transrest'] = df_train['transrest'].apply(classify_transmission)
df_test['transrest'] = df_test['transrest'].apply(classify_transmission)


############################## Adjust the variable types
df_train['accidentbool'] = df_train['accidentbool'].astype('bool')
df_train['cleanbool'] = df_train['cleanbool'].astype('bool')
df_train['brand_type'] = df_train['brand_type'].astype('int')

df_test['accidentbool'] = df_test['accidentbool'].astype('bool')
df_test['cleanbool'] = df_test['cleanbool'].astype('bool')
df_test['brand_type'] = df_test['brand_type'].astype('int')


#### Process Fuel_types
dictfuel = {'special': 5, 'Hybrid':4, 'Diesel': 3, 'Plug-In Hybrid': 4, 
            'Gasoline': 2, 'â€“':1, 'E85 Flex Fuel' : 1, 'not supported': 1}
df_train['fuel_type'] = df_train['fuel_type'].map(dictfuel)
df_train['fuel_type'] = df_train['fuel_type'].fillna(6)
df_train['fuel_type'] = df_train['fuel_type'].astype('int')

df_test['fuel_type'] = df_test['fuel_type'].map(dictfuel)
df_test['fuel_type'] = df_test['fuel_type'].fillna(6)
df_test['fuel_type'] = df_test['fuel_type'].astype('int')

############################## Dropping columns not to use now
df_train.drop(columns=['brand', 'model', 'engine', 'transmission', 'speed', 'int_col', 'ext_col'], inplace=True)
df_test.drop(columns=['brand', 'model', 'engine', 'transmission', 'speed', 'int_col', 'ext_col'], inplace=True)

C:\Users\higaj\AppData\Local\Temp\ipykernel_6140\1200941474.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_train.loc[df_train['accident'] == 'None reported', 'accidentbool'] = False
C:\Users\higaj\AppData\Local\Temp\ipykernel_6140\1200941474.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_test.loc[df_test['accident'] == 'None reported', 'accidentbool'] = False
C:\Users\higaj\AppData\Local\Temp\ipykernel_6140\1200941474.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.


In [2]:
df_train = df_train[df_train['price'] <= 500000]

In [3]:
import optuna

import lightgbm as lgb

from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# Define the Tables
X = df_train
X = X.drop(columns='price')
y = df_train.price

# Separate test and validation of the df_train dataframe
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)



def evaluate_model(model, X_train, X_valid, y_train, y_valid, model_name):
    print(f"Training {model_name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    model_scores.append((model_name, rmse))
    print(f"{model_name} RMSE: {rmse:.4f}")
    return rmse


model_scores = []
best_hyperparams = {}

def objective_lightgbm(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 1.0),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    return rmse

# Define the models and their respective objective functions
model_objectives = {
    "LightGBM": objective_lightgbm,
}

# Optimize each model with Optuna and store the results
for model_name, objective in model_objectives.items():
    print(f"Optimizing {model_name}...")
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100, show_progress_bar=True)
    
    #Store the best result
    model_scores.append((model_name, study.best_value))
    best_hyperparams[model_name] = study.best_params

# Print all model scores
for model_name, rmse in model_scores:
    print(f"{model_name}: RMSE = {rmse:.4f}")

# Optional: Convert model_scores to a DataFrame for better readability
scores_df = pd.DataFrame(model_scores, columns=['Model', 'RMSE']).sort_values(by='RMSE')
print(scores_df)

# Print the best hyperparameters for each model
for model_name, params in best_hyperparams.items():
    print(f"\nBest hyperparameters for {model_name}:")
    print(params)

[I 2024-09-25 15:56:57,683] A new study created in memory with name: no-name-4ce9df3d-2f94-4429-b758-0c90cddde334


Optimizing LightGBM...


  0%|          | 0/100 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l1 is set=0.4348067246733402, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.4348067246733402
[LightGBM] [Warning] lambda_l2 is set=0.5342737726879023, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5342737726879023
[LightGBM] [Warning] lambda_l1 is set=0.4348067246733402, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.4348067246733402
[LightGBM] [Warning] lambda_l2 is set=0.5342737726879023, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5342737726879023
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005124 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 592
[LightGBM] [Info] Number of data points in the train set: 150544, number of used features: 8
[LightGBM] [Info] Start training from score 41480.239053
[LightGBM] [Warning] lambda_l1 is set=0.4348067246733402,

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] lambda_l1 is set=0.9460911305143975, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9460911305143975
[LightGBM] [Warning] lambda_l2 is set=0.9548679771147877, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.9548679771147877
[I 2024-09-25 16:00:45,754] Trial 84 finished with value: 35657.62544980936 and parameters: {'n_estimators': 666, 'max_depth': 9, 'learning_rate': 0.026975640231664454, 'subsample': 0.6196857294655604, 'colsample_bytree': 0.6022901711820252, 'lambda_l2': 0.9548679771147877, 'lambda_l1': 0.9460911305143975, 'min_child_weight': 7}. Best is trial 34 with value: 35647.37695493436.
[LightGBM] [Warning] lambda_l1 is set=0.9191638897328454, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9191638897328454
[LightGBM] [Warning] lambda_l2 is set=0.8090067022383974, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.8090067022383974
[LightGBM] [Warning] lambda_l1 is set=0.9191638897328454, reg_alpha=0.0 will be ignore

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011701 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 592
[LightGBM] [Info] Number of data points in the train set: 150544, number of used features: 8
[LightGBM] [Info] Start training from score 41480.239053
[LightGBM] [Warning] lambda_l1 is set=0.9976435121544425, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9976435121544425
[LightGBM] [Warning] lambda_l2 is set=0.5119096392795179, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5119096392795179
[I 2024-09-25 16:01:05,093] Trial 92 finished with value: 35686.468497028894 and parameters: {'n_estimators': 223, 'max_depth': 9, 'learning_rate': 0.037270706913775944, 'subsample': 0.6088662932788853, 'colsample_bytree': 0.609646071733336, 'lambda_l2': 0.5119096392795179, 'lambda_l1': 0.9976435121544425, 'min_child_weight': 2}. Best is trial 34 with value: 35647.37695493436.
[LightGB

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] lambda_l1 is set=0.6893964905782651, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.6893964905782651
[LightGBM] [Warning] lambda_l2 is set=0.4562123461787502, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.4562123461787502
[I 2024-09-25 16:01:15,248] Trial 97 finished with value: 35663.636897991 and parameters: {'n_estimators': 403, 'max_depth': 8, 'learning_rate': 0.024438950552534623, 'subsample': 0.5321347481104315, 'colsample_bytree': 0.5758590166979701, 'lambda_l2': 0.4562123461787502, 'lambda_l1': 0.6893964905782651, 'min_child_weight': 3}. Best is trial 34 with value: 35647.37695493436.
[LightGBM] [Warning] lambda_l1 is set=0.6262439572349037, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.6262439572349037
[LightGBM] [Warning] lambda_l2 is set=0.42097734351948624, reg_lambda=0.

In [6]:
params = {'n_estimators': 386, 'max_depth': 10, 'learning_rate': 0.031130813222088233, 'subsample': 0.5046037919145264, 'colsample_bytree': 0.6329533186063119, 'lambda_l2': 0.733873227919962, 'lambda_l1': 0.9944125775497417, 'min_child_weight': 2}
model = lgb.LGBMRegressor(**params)
#model = lgb.LGBMRegressor()
# Define the Tables
X = df_train
X = X.drop(columns='price')
y = df_train.price

# Separate test and validation of the df_train dataframe
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)




model.fit(X_train, y_train)
y_pred = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))

print(f"{model}: RMSE = {rmse:.4f}")

submission = pd.DataFrame()
temp = pd.read_csv(r'C:\Users\higaj\Desktop\py\used_cars\test.csv')
submission['id'] = temp['id']
submission['price'] = prediccion
submission.to_csv('submission5.csv',index=False)

[LightGBM] [Warning] lambda_l1 is set=0.9944125775497417, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9944125775497417
[LightGBM] [Warning] lambda_l2 is set=0.733873227919962, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.733873227919962
[LightGBM] [Warning] lambda_l1 is set=0.9944125775497417, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9944125775497417
[LightGBM] [Warning] lambda_l2 is set=0.733873227919962, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.733873227919962
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008530 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 592
[LightGBM] [Info] Number of data points in the train set: 150544, number of used features: 8
[LightGBM] [Info] Start training from score 41480.239053
[LightGBM] [Warning] lambda_l1 is set=0.9944125775497417, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.994412577

In [ ]:
df_train.loc[df_train['milage']>250000]